In [99]:
import pandas as pd
from pathlib import Path

In [100]:
assert_labels = '../data/raw_data/assert_ade/test_assert_ade_smm4h_task1.csv'
ner_labels = '../data/raw_data/ner_ade/test_ner_ade_smm4h_task2.csv'

df_assert_gold = pd.read_csv(assert_labels)
df_ner_gold = pd.read_csv(ner_labels)

In [101]:
submission = Path('../predictions_val/')

In [102]:
df_ner = []
df_assert = []
for file in submission.iterdir():
    df = pd.read_csv(file / 'test_predictions.csv')
    if file.name.startswith('ner'):
        df_ner.append(df.assign(model=file.name.replace('ner_', '')))
    elif file.name.startswith('assert'):
        df_assert.append(df.assign(model=file.name.replace('assert_', '')))
df_ner = pd.concat(df_ner)
df_assert = pd.concat(df_assert)

## Classification Errors

In [103]:
df_assert_gold['label'] = df_assert_gold['target_text'].apply(lambda t: 'NoADE' if t == 'healthy okay' else 'ADE')

In [104]:
df_assert['class'] = df_assert['predicted_text'].apply(lambda t: 'NoADE' if t == 'healthy okay' else 'ADE')

In [105]:
df_assert = pd.merge(df_assert, df_assert_gold[['tweet_id', 'label']], on='tweet_id', how='left')

### False positives

In [107]:
df_assert_fp = df_assert[(df_assert['class'] == 'ADE') & (df_assert['label'] == 'NoADE')]
df_assert_fp[['model', 'tweet_id',  'tweet', 'class', 'label']].to_csv('assert_ade_fp.csv')
df_assert_fp.groupby('model').count()

,tweet_id,predicted_text,counts,tweet,prefix,input_text,target_text,class,label
model,,,,,,,,,
ade_ensemble_mv_1,11,11,11,11,0,0,0,11,11
ade_markov_1,17,17,0,17,17,17,17,17,17
ade_raw_1,15,15,0,15,15,15,15,15,15
ade_ri_1,9,9,0,9,9,9,9,9,9
ade_sr_1,6,6,0,6,6,6,6,6,6
ade_wnet_1,11,11,0,11,11,11,11,11,11
over_under_1,19,19,0,19,19,19,19,19,19


### False negatives

In [109]:
df_assert_fn = df_assert[(df_assert['label'] == 'ADE') & (df_assert['class'] == 'NoADE')]
df_assert_fn[['model', 'tweet_id',  'tweet', 'class', 'label']].to_csv('assert_ade_fn.csv')
df_assert_fn.groupby('model').count()

,tweet_id,predicted_text,counts,tweet,prefix,input_text,target_text,class,label
model,,,,,,,,,
ade_ensemble_mv_1,18,18,18,18,0,0,0,18,18
ade_markov_1,17,17,0,17,17,17,17,17,17
ade_raw_1,20,20,0,20,20,20,20,20,20
ade_ri_1,23,23,0,23,23,23,23,23,23
ade_sr_1,20,20,0,20,20,20,20,20,20
ade_wnet_1,20,20,0,20,20,20,20,20,20
over_under_1,9,9,0,9,9,9,9,9,9


## NER Errors

In [111]:
df_ner['predicted_text'] = df_ner['predicted_text'].apply(lambda s: s.strip('.'))

In [112]:
df_ner_w_gold = pd.merge(df_ner, df_ner_gold[['tweet_id', 'target_text']], on='tweet_id', how='left')

In [128]:
df_ner_ne = df_ner_w_gold[df_ner_w_gold['target_text_y'] != df_ner_w_gold['predicted_text']]
df_ner_ne.groupby('model')['tweet_id'].count().reset_index()

,model,tweet_id
0,ensemble_over_under_1_prompt2,83
1,ensemble_prompt,44
2,ensemble_ri_1_over_under_1_prompt2,47
3,ensemble_test,47
4,over_under_1,64
5,prompt1,47
6,prompt2,46
7,prompt3,50
8,ri_1,52
9,sr_1,50


### False negatives

In [129]:
df_ner_fn = df_ner_w_gold[(df_ner_w_gold['target_text_y'] != 'none') & (df_ner_w_gold['predicted_text'] == 'none')]
df_ner_fn.to_csv('ner_ade_fn.csv')
df_ner_fn.groupby('model')['tweet_id'].count().reset_index()

,model,tweet_id
0,ensemble_over_under_1_prompt2,14
1,ensemble_prompt,11
2,ensemble_ri_1_over_under_1_prompt2,7
3,ensemble_test,13
4,over_under_1,9
5,prompt1,17
6,prompt2,10
7,prompt3,12
8,ri_1,15
9,sr_1,25


In [136]:
df_ner_fn[df_ner_fn['model'] == 'ensemble_over_under_1_prompt2']['tweet'].iloc[0]

"@USER___________ I've had Cipro before. Luckily for me, the only side fx I tend to get from AB is gastic upset. But I RARELY use AB."

## False positives

In [130]:
df_ner_fp = df_ner_w_gold[(df_ner_w_gold['target_text_y'] == 'none') & (df_ner_w_gold['predicted_text'] != 'none')]
df_ner_fp.to_csv('ner_ade_fp.csv')
df_ner_fp.groupby('model')['tweet_id'].count().reset_index()

,model,tweet_id
0,ensemble_over_under_1_prompt2,43
1,ensemble_prompt,14
2,ensemble_ri_1_over_under_1_prompt2,21
3,ensemble_test,18
4,over_under_1,40
5,prompt1,16
6,prompt2,15
7,prompt3,23
8,ri_1,23
9,sr_1,12


In [135]:
df_ner_fp[df_ner_fp['model'] == 'ensemble_over_under_1_prompt2']['tweet'].iloc[0]

"@USER_____ bloody hell.  i'm on high dose venlafaxine and have been for years so, yay mentals! \\m/ #mentalhealth"

### Partial or no match

In [131]:
df_ner_partial = df_ner_w_gold[(df_ner_w_gold['target_text_y'] != 'none') & (df_ner_w_gold['predicted_text'] != 'none') & (df_ner_w_gold['target_text_y'] != df_ner_w_gold['predicted_text'])]
df_ner_partial.to_csv('ner_ade_partial.csv')
df_ner_partial.groupby('model')['tweet_id'].count().reset_index()

,model,tweet_id
0,ensemble_over_under_1_prompt2,26
1,ensemble_prompt,19
2,ensemble_ri_1_over_under_1_prompt2,19
3,ensemble_test,16
4,over_under_1,15
5,prompt1,14
6,prompt2,21
7,prompt3,15
8,ri_1,14
9,sr_1,13


In [142]:
df_ner_partial[df_ner_partial['model'] == 'ensemble_over_under_1_prompt2']['tweet'].iloc[0], \
df_ner_partial[df_ner_partial['model'] == 'ensemble_over_under_1_prompt2']['predicted_text'].iloc[0], \
df_ner_partial[df_ner_partial['model'] == 'ensemble_over_under_1_prompt2']['target_text_y'].iloc[0], \

('"trying to distract with masterchef, twitter, texting &amp; candy crush but i\'m anxious &amp; have stomach &amp; head pain #venlafaxine #withdrawal"',
 'stomach',
 'anxious')